# Pré processamento
Agora que filtramos nossos dados, queremos adicionar colunas que podem ser inferidas através dos atributos originais.

In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
# Arquivos ainda não processados

users = pd.read_csv('original_data/users_cleaned.csv')
animes = pd.read_csv('original_data/anime_cleaned.csv')
user_animes = pd.read_csv('original_data/user_anime.csv')

In [4]:
# Adicionando coluna de idade nos usuários
users['age'] = users['birth_date'].apply(lambda x: 2018 - int(x.split("-")[0]))

In [6]:
# Criando dicionário de países e regiões

with open('countries.json') as f:
    data = json.load(f)
names = []
for i in data:
    names.append(i['name'])
dictnames = dict();
for i in names:
    dictnames[i['common']] = [i['common']]
    dictnames[i['common']].append(i['official'])
    for n in [i['native']]:
        for j in n:
            dictnames[i['common']].append(n[j]['common'])
            dictnames[i['common']].append(n[j]['official'])
alternativeus = ['US', 'USA', 'U.S.', 'U.S', 'U.S.A.', 'America']
for i in alternativeus:
    dictnames['United States'].append(i)
dictnames['United Kingdom'].append('UK')
dictnames['United Kingdom'].append('England')
dictnames['Canada'].append('Montreal')
dictnames['Canada'].append('Montréal')

In [7]:
# Adicionando estados americanos no dicionário

states = pd.read_csv('states.csv')
states = states.to_dict("records")
for i in states:
    dictnames['United States'].append(i['State'])
    dictnames['United States'].append(i['Abbreviation'])

In [9]:
# Adicionando coluna de regiões, limpando elementos não desejados

def findCountry(x):
    global c
    c = c + 1
    if not isinstance(x, int) and not isinstance(x, float):
        for i in dictnames:
            for j in dictnames[i]:
                if j in x: return i
        return "no data"
    else: return "no data"

total = len(users.index)
c = 0
users['country'] = users['location'].apply(lambda x: findCountry(x))
print("Done!")

Done!


In [10]:
# Realizando merge nas tabelas User, Anime e UserAnime
merged = users.merge(user_animes, left_on='username', right_on='username', how='inner')
merged = merged.merge(animes, left_on='anime_id', right_on='anime_id', how='inner')

In [15]:
# Adicionanod coluna de porcentagem de completude usuário-anime
merged['my_completion'] = round((merged['my_watched_episodes']/merged['episodes']),2)

In [12]:
# Transformando dados pré-processados em CSV para leitura em outros arquivos
users.to_csv('manipulated_data/users.csv')